In [ ]:
from pathlib import Path 
from bs4 import BeautifulSoup
import json
import pandas as pd 
import re 
from lingua import Language, LanguageDetectorBuilder
from datetime import datetime
import os

In [ ]:
## putting the processed entire thread data together
directory = "redata"
output_file = 'raw_merged_sep22_jun23.txt'
content_list = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            content_list.append(content)

merged_content = '\n\n'.join(content_list)

with open(output_file, 'w', encoding='utf-8') as output:
    output.write(merged_content)

In [ ]:
def read_file_as_string(file_path):
    text = Path(file_path).read_text(encoding='utf-8')
    return text

filepath = '/Users/emilyzou/Desktop/pol/raw_merged_sep22_jun23.txt'
text = read_file_as_string(filepath)

In [ ]:
def return_json (input_string): 
    html = input_string
    soup = BeautifulSoup(html, 'html.parser')
    soup = BeautifulSoup(input_string, 'html.parser')
    posts_data = []
    for post_wrapper in soup.find_all("div", class_="post_wrapper"):
        post_data = {
            "post_data": {
                "author": post_wrapper.find("span", class_="post_author").text.strip() if post_wrapper.find("span", class_="post_author") else "",
                "tripcode": post_wrapper.find("span", class_="post_tripcode").text.strip() if post_wrapper.find("span", class_="post_tripcode") else "",
                "poster_hash": post_wrapper.find("span", class_="poster_hash").text.strip() if post_wrapper.find("span", class_="poster_hash") else "",
                "datetime": post_wrapper.find("time")["datetime"] if post_wrapper.find("time") else "",
                "time_text": post_wrapper.find("time").text.strip() if post_wrapper.find("time") else "",
                "post_id": post_wrapper.find("a", {"data-function": "quote"})["data-post"] if post_wrapper.find("a", {"data-function": "quote"}) else "",
                "post_link": post_wrapper.find("a", {"data-function": "quote"})["href"] if post_wrapper.find("a", {"data-function": "quote"}) else ""
            },
            "controls": {
                "controls_links": [
                    control.get("href", "#") for control in post_wrapper.select(".post_controls a")
                ]
            },
            "backlink_list": {
                "quoted_by": [
                    backlink["data-post"] for backlink in post_wrapper.select(".post_backlink")
                ]
            },
            "text_content": {
                "text": post_wrapper.find("div", class_="text").get_text(separator="\n").strip() if post_wrapper.find("div", class_="text") else "",
                "greentext_links": [
                    link["href"] for link in post_wrapper.find_all("a", class_="backlink") if link
                ]
            }
        }
        posts_data.append(post_data)
    json_output = json.dumps(posts_data, indent=4)
    return json_output

In [ ]:
with open('all_articles.json', 'r') as file:
    json_literal_string = file.read()
data = json.loads(json_literal_string)
df = pd.DataFrame([(key, item) for key, values in data.items() for item in values], columns=['Thread', 'Content'])
df['ID'] = df['Content'].str.extract(r'^\S+ ID:([\w\d]+)')
df['No'] = df['Content'].str.extract(r'\bNo\.(\d+)\b')
df['Date'] = df['Content'].str.extract(r'\b(\w{3} \d{1,2} \w{3} \d{4} \d{2}:\d{2}:\d{2})\b')
df['Literal'] = df['Content'].apply(repr)
df['LosingIt'] = df['Literal'].str.split('Report', n=1).str[1]
df['OnlyQuotes'] = df['LosingIt'].apply(lambda x: x if 'Quoted By' in x else [])
df['Quoted'] = df['OnlyQuotes'].apply(lambda x: re.search(r'(\\n.*?\\n)', x).group(0) if x.count('\\n') > 1 else "")
df['Quotes'] = df['Quoted'].apply(lambda x: re.findall(r'>>(\d+)', x))
df['Reply1'] = df.apply(lambda row: row['LosingIt'].replace(row['Quoted'], ''), axis=1)
df['Replies'] = df['Reply1'].apply(lambda x: re.findall(r'>>(\d+)', x))
df['Text'] = df['Content'].apply(lambda x: x.split("\n")[-1])

In [ ]:
adf = df[['Thread', 'ID', 'No', 'Date', 'Quotes', 'Replies', 'Text']]

In [ ]:
adflist = [m for m in adf['No']]
root_id_list = [m for m in root_dei if m in adflist]
